In [1]:
from ResNet import ResNet18, ResNet34, ResNet50, ResNet101, ResNet152
from torchvision import datasets, models
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import torch
import torchvision
import os
import time

import matplotlib.pyplot as plt
import numpy as np

In [ ]:
def create_model():
    model = ResNet18(pretrained=True)
    model.fc = nn.Linear(model.fc.in_features, 10)

    model = model.to(device)
    return model

def dataset_loaders():
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
    
    train_set = datasets.CIFAR10(root='./data/', train=True, download=True, transform=transform)
    test_set = datasets.CIFAR10(root='./data/', train=False, download=True, transform=transform)
    
    train_loader = torch.utils.data.DataLoader(dataset=train_set, batch_size=32, num_workers=8, shuffle=True)
    test_loader = torch.utils.data.DataLoader(dataset=test_set, batch_size=32, num_workers=8, shuffle=True)
    print(f"{train_set.classes}")
    return (train_loader, test_loader)

def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()
    
def show_predicts(images, labels):
    imshow(torchvision.utils.make_grid(images))
    print('Predicted: ', [torch.argmax(x) for x in labels])

In [2]:
model = ResNet18(pretrained=True)

pretrained


In [ ]:
%%time 
train_loader, test_loader = dataset_loaders()
data_iterator = iter(test_loader)
images, labels = next(data_iterator)
show_predicts(images, labels)

In [ ]:
def train(model, criterion, optimizer, dataloaders, epochs=25):
    start_time = time.time()
    
    best_acc = 0.00
    
    datasize = dict()
    datasize['train'], datasize['val'] = len(dataloaders['train']), len(dataloaders['val'])
    
    for epoch in range(epochs):
        print(f'Epoch {epoch}/{epochs-1}')
        
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()
                
            loss = 0.00
            corrects = 0
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
            
                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    predicted = torch.argmax(outputs)
                    
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                
                loss += loss.item() * inputs.size(0)
                corrects += torch.sum(predicted == labels.data)
            
            epoch_loss = loss / datasize[phase]
            epoch_acc = corrects/ datasize[phase]
            
            if epoch_acc > best_acc:
                best_acc = epoch_acc
            
            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
            
    time_elapsed = time.time() - start_time
    print('Training complete in {time_elapsed:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print(f'Best val Acc: {best_acc:4f}')

In [ ]:
model = create_model()

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

dataloaders = dict()
dataloaders['train'], dataloaders['val'] = dataset_loaders()

criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(model.parameters(), lr=1e-3, momentum=0.9)

train(model, criterion, optimizer, dataloaders)